## Load in packages and modified FROMAGe

In [18]:
%cd ../..

c:\Users\20175862\Documents\MscArtificialIntelligence\DL2


In [4]:
from src.fromage_inf.inf_utils import PromptParser
import pickle
from random import choice

c:\Users\20175862\Miniconda3\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: [WinError 127] The specified procedure could not be found
  warn(f"Failed to load image Python extension: {e}")


In [6]:
parser = PromptParser("src/fromage_inf/fromage_model/")

# Load the relations dictionary to make the relations.
relations_dict = pickle.load(open("src/code/relations_dict.pkl", "rb"))

In [ ]:
model = parser.model
relations = model.visual_embs

## Few-shot Chain-of-thought arithmetics

In [17]:
EXAMPLES = 3
for relation, values in relations_dict.items():

    print(relation)
    print('=' * 120)
    for i in range(0, len(values)):
        print(values)
        tuple1 = values[i]
        
        if i == len(values)-1:
            tuple2 = values[0]
        else:
            tuple2 = values[i+1]
        
        print('='*60)
        print("Arithmetic:\n")

        print("Few-shot examples:\n")

        extra_examples = []
        not_allowed = []
        for _ in range(min(EXAMPLES, len(values)-1)): 
            # if there are less relations in a category than the specified EXAMPLES, use #relations -1 instead
            # eg for food there are only 3 relations -> use 2 examples instead of 3 if EXAMPLES is set to 3
            not_allowed = extra_examples + [i]
            print('not_allowed',not_allowed)
            extra_examples.append(choice([j for j in range(0,len(values)) if j not in not_allowed])) 
            
            print('extra_examples',extra_examples)
            # choice selects from all instances in the current relation class
            # excluding already used examples and the prediction pair

        prefix = []
        for ex in extra_examples: # add extra examples to prompt prefix
            example_tuple1 = values[ex]

            example_tuple2_idx = choice([j for j in range(0,len(values)) if j not in [ex, i]])
            example_tuple2 = values[example_tuple2_idx]
            
            prefix += [model.visual_embs[example_tuple1[1]] + (model.visual_embs[example_tuple2[0]] - model.visual_embs[example_tuple2[1]]), ' -> ', model.visual_embs[example_tuple1[0]]] # add correct output image

            print("{} + ({} - {}) -> {}".format(example_tuple1[1], example_tuple2[0], example_tuple2[1], example_tuple1[0]))
        
        print("{} + ({} - {})".format(tuple1[1], tuple2[0], tuple2[1]))
        print("Expected result: {}".format(tuple1[0]))

        inp_image = model.visual_embs[tuple1[1]] + (model.visual_embs[tuple2[0]] - model.visual_embs[tuple2[1]])
        prompt = prefix + [inp_image, "[RET]"]

        print('=' * 30)
        model_outputs = model.generate_for_images_and_texts(prompt, max_img_per_ret=3)

        print('Model generated outputs:')
        parser.display(model_outputs)

companies -> CEOs
[('companies/facebook', 'CEOs/mark_zuckerberg'), ('companies/microsoft', 'CEOs/bill_gates'), ('companies/tesla', 'CEOs/elon_musk'), ('companies/amazon', 'CEOs/jeff_bezos'), ('companies/apple', 'CEOs/steve_jobs')]
Arithmetic:

Few-shot examples:

not_allowed [0]
extra_examples [3]
not_allowed [3, 0]
extra_examples [3, 1]
not_allowed [3, 1, 0]
extra_examples [3, 1, 2]
CEOs/jeff_bezos + (companies/microsoft - CEOs/bill_gates) -> companies/amazon
CEOs/bill_gates + (companies/apple - CEOs/steve_jobs) -> companies/microsoft
CEOs/elon_musk + (companies/amazon - CEOs/jeff_bezos) -> companies/tesla
CEOs/mark_zuckerberg + (companies/microsoft - CEOs/bill_gates)
Expected result: companies/facebook
Model generated outputs:
[('companies/facebook', 'CEOs/mark_zuckerberg'), ('companies/microsoft', 'CEOs/bill_gates'), ('companies/tesla', 'CEOs/elon_musk'), ('companies/amazon', 'CEOs/jeff_bezos'), ('companies/apple', 'CEOs/steve_jobs')]
Arithmetic:

Few-shot examples:

not_allowed [1]